In [ ]:
propno = 3118
runno = 18

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

from collections import namedtuple
from extra_data import RunDirectory
from extra_data.read_machinery import find_proposal
from extra_redu import make_litframe_finder, ReferenceDelay, FrameSelection
from extra_redu.litfrm.utils import (
    find_ranges_of_patterns, compress_ranges_of_patterns)
from extra_redu.litfrm.draw.agipd import draw_cells

In [ ]:
propdir = find_proposal(f"p{propno:06d}")
rundir = os.path.join(propdir, f"raw/r{runno:04d}")

run = RunDirectory(rundir)
inst = rundir.split('/')[4]

In [ ]:
dev = make_litframe_finder(inst, run)

In [ ]:
tm0 = time.monotonic()
r = dev.read_or_process()
tid = r.meta.trainId
tm1 = time.monotonic()

print(f"N trains: {tid.size}, time: {tm1-tm0:.2f}s, train per sec: {tid.size/(tm1-tm0):.2f}")

In [ ]:
ptrn, ix, nptrn = np.unique(r.output.nPulsePerFrame, axis=0, return_counts=True, return_inverse=True)

In [ ]:
ranges = find_ranges_of_patterns(ptrn.shape[0], ix, tid)
for a, rng_list in ranges.items():
    print(a, rng_list)

In [ ]:
cranges = compress_ranges_of_patterns(ranges, tid)
for a, rng_list in cranges:
    print(a, rng_list)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16,4))
for a, rng_list in cranges:
    for t0, tN, s, ntrn, nmis in rng_list:
        t = np.arange(t0, tN, s, dtype=np.uint64)
        g = np.in1d(t, tid)
        ax.plot(t[g], [a] * g.sum(), 'C0.')
        ax.plot(t[~g], [a] * (~g).sum(), 'C1o')
        
plt.show()

In [ ]:
sel = FrameSelection(r, guess_missed=True)

print(" # trains                     Ntrn Nmis   Np  Nd  Nf lit frames")
for rec in sel.report():
    frmintf = ', '.join(
        [':'.join([str(n) for n in slc]) for slc in rec['litframe_slice']]
    )
    t0, tN, st = (rec['train_range'] + (1,))[:3]
    ntrain = max((int(tN) - int(t0)) // int(st), 1)
    trsintf = ':'.join([str(n) for n in rec['train_range']])
    print(
        ("{pattern_no:2d} {trsintf:25s} {ntrain:5d} {nmissed_trains:4d} "
         "{npulse_exposed:4d} {ndataframe:3d} {nframe_total:3d} [{frmintf}]"
        ).format(frmintf=frmintf, ntrain=ntrain, trsintf=trsintf, **rec)
    )
    i = np.flatnonzero(rec['train_range'][0] == tid)[0]